In [1]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.preprocessing import StandardScaler
import pandas as pd

In [2]:
def load_data():
    cols_91 = ['Year','Month','DayofMonth','DayOfWeek','DepTime','CRSDepTime','ArrTime','CRSArrTime','UniqueCarrier','FlightNum','ActualElapsedTime','CRSElapsedTime','ArrDelay','DepDelay','Origin','Dest','Distance','Cancelled','Diverted']
    cols_01 = ['Year','Month','DayofMonth','DayOfWeek','DepTime','CRSDepTime','ArrTime','CRSArrTime','UniqueCarrier','FlightNum','TailNum','ActualElapsedTime','CRSElapsedTime','ArrDelay','DepDelay','Origin','Dest','Distance','TaxiIn','TaxiOut','Cancelled','Diverted']
    return pd.read_csv("Data/1991.csv.gz", encoding='cp1252', compression="gzip", usecols=cols_91), pd.read_csv("Data/2001.csv.gz", encoding='cp1252', compression="gzip", usecols=cols_01)

In [3]:
df91, df01 = load_data()

In [4]:
df91.sample(10)

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,ActualElapsedTime,CRSElapsedTime,ArrDelay,DepDelay,Origin,Dest,Distance,Cancelled,Diverted
3279358,1991,8,25,7,700.0,700,903.0,910,ML (1),394,123.0,130,-7.0,0.0,DFW,MDW,797.0,0,0
4144918,1991,10,22,2,1801.0,1800,1856.0,1855,NW,295,115.0,115,1.0,1.0,DTW,MEM,610.0,0,0
4566037,1991,11,28,4,902.0,900,1334.0,1339,AA,296,152.0,159,-5.0,2.0,LAS,DFW,1055.0,0,0
1063901,1991,3,19,2,1604.0,1600,1734.0,1747,DL,883,90.0,107,-13.0,4.0,CLE,ATL,554.0,0,0
1797612,1991,5,21,2,1325.0,1330,1456.0,1515,NW,1444,91.0,105,-19.0,-5.0,DTW,PVD,614.0,0,0
339695,1991,1,5,6,NaN,1116,NaN,1237,AA,516,NaN,81,NaN,NaN,MCI,ORD,403.0,1,0
777586,1991,2,10,7,1345.0,1346,1824.0,1814,CO,168,219.0,208,10.0,-1.0,IAH,LGA,1416.0,0,0
4897890,1991,12,10,2,2217.0,2218,2322.0,2320,DL,641,65.0,62,2.0,-1.0,DFW,LBB,282.0,0,0
2413216,1991,6,23,7,921.0,920,1049.0,1052,NW,317,148.0,152,-3.0,1.0,DCA,MSP,931.0,0,0
2061448,1991,5,23,4,1904.0,1904,2232.0,2216,CO,170,148.0,132,16.0,0.0,IAH,MCO,853.0,0,0


In [41]:
df = df91

In [42]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
import pandas as pd

# Assuming 'df' is your DataFrame containing the data

# Handling missing values
df = df.dropna()  # Dropping rows with missing values or use df.fillna() to impute missing values

df['OnTime'] = df['ArrDelay'].apply(lambda x: 1 if x <= 0 else 0)

# Separate features and the modified target variable
X = df.drop(columns=['ArrDelay', 'OnTime'])  # Features
y = df['OnTime']  # Target variable

# Select categorical columns
categorical_cols = ['UniqueCarrier', 'Origin', 'Dest']

# Select non-categorical columns
non_categorical_cols = [col for col in X.columns if col not in categorical_cols]

# Handle missing values in the numerical columns (if any)
numerical_cols = non_categorical_cols
imputer = SimpleImputer(strategy='mean')
X[numerical_cols] = imputer.fit_transform(X[numerical_cols])

# Apply Label Encoding to categorical columns
encoder = LabelEncoder()
X_encoded = X[categorical_cols].apply(encoder.fit_transform)

# Combine encoded and non-categorical columns
X_processed = pd.concat([X_encoded.reset_index(drop=True), X[numerical_cols].reset_index(drop=True)], axis=1)

# Splitting the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=42)

# Training the Random Forest Regression model
rf_reg = RandomForestRegressor(n_estimators=100, random_state=42)
rf_reg.fit(X_train, y_train)

# Extracting feature importance
feature_importance = pd.DataFrame({
    'Feature': X_processed.columns,
    'Importance': rf_reg.feature_importances_
})

# Sorting features by importance
feature_importance = feature_importance.sort_values(by='Importance', ascending=False)

# Displaying the most important features
print(feature_importance)


C:\Users\Mubarak\AppData\Local\Temp\ipykernel_11272\1988713493.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['OnTime'] = df['ArrDelay'].apply(lambda x: 1 if x <= 0 else 0)


              Feature  Importance
13     CRSElapsedTime    0.363123
12  ActualElapsedTime    0.348023
14           DepDelay    0.283620
11          FlightNum    0.000638
9             ArrTime    0.000603
5          DayofMonth    0.000580
15           Distance    0.000532
1              Origin    0.000447
2                Dest    0.000429
7             DepTime    0.000429
4               Month    0.000372
10         CRSArrTime    0.000348
6           DayOfWeek    0.000314
8          CRSDepTime    0.000305
0       UniqueCarrier    0.000238
3                Year    0.000000
16          Cancelled    0.000000
17           Diverted    0.000000
